In [2]:
%cd /src/notebooks

/src/notebooks


In [4]:
!pip install -r ../libs/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 2.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.2 MB/s eta 0:00:00a 0:00:01


In [5]:
from libs.dataset_generator.generator import generator
from libs.dataset_generator.exporter import DatasetExporterHuggingFace

import dotenv
dotenv.load_dotenv()

True

In [6]:
dataset = await generator.generate()
print(len(dataset))
await DatasetExporterHuggingFace(
    dataset=dataset,
    repo_id = "LeonardoBenitez/capivara-plugin-orchestration",
    dataset_config='plugins_capital_v1.0',
).export()

#await DatasetExporterLocal(
#    dataset=dataset,
#    path_local_base = '/src/data/plugins_capital_v1.0',
#).export()
dataset

[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 0 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 1 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 2 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 3 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 4 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 5 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 6 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 7 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 8 for case 2b
[2024-08-17 13:20:18 +0000] libs.libs.dataset_generator INFO     Generated trajectory 9 for case 2b


tmp3vomittx.parquet:   0%|          | 0.00/816k [00:00<?, ?B/s]

[2024-08-17 13:21:19 +0000] libs.libs.dataset_generator INFO     Uploaded train with 8000 rows


tmp7rhxbuat.parquet:   0%|          | 0.00/99.8k [00:00<?, ?B/s]

[2024-08-17 13:21:20 +0000] libs.libs.dataset_generator INFO     Uploaded validation with 1000 rows


tmpl7w546r_.parquet:   0%|          | 0.00/103k [00:00<?, ?B/s]

[2024-08-17 13:21:22 +0000] libs.libs.dataset_generator INFO     Uploaded test with 1000 rows


[[ChatCompletionMessage(role='system', content="You are allowed to call the following function:\n1. PluginCapital_get_capital: Returns the name of the capital of a country..\n  Arguments: country (Name of the country. Type: string. Required: yes)\n2. answer: Send response back to the user. Show all your results here, this is the only thing that the user will see. You won't be able to call any other function after this one. Be sure to return a complete and clear answer, the user will not be able to see any other intermediate messages nor ask for more information. Never mention intermediate messages or results; if you want to mention something, include it here. Call this function only once, with everything you want to show to the user..\n  Arguments: text (final textual response to be send to the user. Should use HTML syntax for formatting. Type: string. Required: yes)\n"),
  ChatCompletionMessage(role='user', content='I have 3 minutes to find out what the capital of Cape verde is. Can y

In [ ]:
'''
trajectory = dataset[15]
exporter = DatasetExporterHuggingFace(
    dataset=dataset,
    repo_id = "LeonardoBenitez/capivara-plugin-orchestration",
    dataset_config='plugins_capital_v1.0',
)
s = exporter._convert_trajectory_to_str(trajectory)
print(s)
''';

In [ ]:
# test
from datasets import load_dataset

ds = load_dataset("LeonardoBenitez/capivara-plugin-orchestration", 'plugins_capital_v1.0')
ds['train'].to_pandas()

In [ ]:
print(ds['train'].to_pandas().iloc[0]['text'])